# 1 - install dependencies

In [1]:
import numpy as np
import cv2 
import os
import mediapipe as mp
import tensorflow as tf 
from tensorflow import keras
import matplotlib.pyplot as plt
import time
import pandas as pd

# 2 - keypoints extractions and drawing

In [2]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

num_hand_marks = 21
num_pose_marks = 33


# holistic model process image and return the results as keypoints
def mediapipe_detection(image,model):
    image  = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image  = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
    return image,results
    
def draw_styled_landmarks(image,results):
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 
    

# read the keypoints and extract them and process them
def extract_keypoints(results):
    # extract pose marks
    if results.pose_landmarks:
        pose = np.array([ [res.x,res.y,res.z,res.visibility] for res in results.pose_landmarks.landmark ]).flatten()
    else:
        pose = np.zeros(num_pose_marks*4)
    
    # extract left hand
    if results.left_hand_landmarks:
        left_hand = np.array([ [res.x,res.y,res.z] for res in results.left_hand_landmarks.landmark ]).flatten()
    else:
        left_hand = np.zeros(num_hand_marks*3)
        
        
    # extract right hand
    if results.right_hand_landmarks:
        right_hand = np.array([ [res.x,res.y,res.z] for res in results.right_hand_landmarks.landmark ]).flatten()
    else:
        right_hand = np.zeros(num_hand_marks*3)
    
    return np.concatenate([pose,left_hand,right_hand])
    

# 3 - read and process data

### 3.1 collect data that in classes 0,1,2,3,4, and available in the data/train directory

In [23]:
data_path = "../../data"
    
data_train_path = os.path.join(data_path,"my_data")

actions = list(os.listdir(data_train_path))
n_actions = len(actions)
dic={}
counter=0
for action in actions:
    dic[action] = counter
    counter+=1

In [26]:
x=[]
y=[]
for action in actions:
    videos = os.listdir(os.path.join(data_train_path,action))
    for video in videos:
        x.append(os.path.join(data_train_path,action,video))
        y.append(dic[action])

train = x
labels = y

In [27]:
print(len(x),len(y))

250 250


### 3.2 process videos into frames and label to cateogries

In [28]:
def capture_frames(video_path):
    video = cv2.VideoCapture(video_path)
    video_length = int(video.get(cv2.CAP_PROP_FRAME_COUNT)) - 1
    # count = 0
    # frame_rate = video.get(cv2.CAP_PROP_FRAME_COUNT)

    count=0
    frames = []
    while video.isOpened():
      # frame_id = video.get(0)
        ret, frame = video.read()
        
        if not ret:
            continue
        frames.append(frame)
        count += 1
        # If there are no more frames left
        if (count > (video_length-1)):
          # Release the feed
          video.release()
        # if frame_id % math.floor(1) == 0:
        #     file_name = f"{video_path}_frame_{count}.jpg"
        #     count += 1
        #     # print(f"File name: {file_name}")
        #     frames.append(frame)
        #     # cv2.imwrite(file_name, frame)
    video.release()
    return np.array(frames)


def get_frames(video_path,num_frames):
    frames = capture_frames(video_path)
    video_length = len(frames)
    steps = video_length/num_frames
    count=0
    new_frames=[]
    while count<video_length:
        frame = frames[int(count)]
#         frame = cv2.resize(frame, (256, 256))
#         frame =  (frame/255.0).astype(np.float16)
        new_frames.append(frame)
        count+=steps

    s = np.array(new_frames[:num_frames])
    print("SHAPE",s.shape)
    return np.array(new_frames[:num_frames])


In [29]:
def extract_keypoints_video(path,i=-1):
    if(i):
        print(i,end="\t")
    frames = get_frames(path,20)
    output_key_points=[]
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        for frame in frames:
            image, results = mediapipe_detection(frame, holistic)
            output_key_points.append(extract_keypoints(results))
    return np.array(output_key_points)
    

### 3.3 - extract the keypoints and save them in train

In [30]:
if "extracted_features.npy" in os.listdir():
    train = np.load("extracted_features.npy")
else:
    train = np.array([extract_keypoints_video(path,f'iteration : {i+1}/{len(x)}') for i,path in enumerate(x) ])
    np.save('extracted_features.npy', train) 

iteration : 1/250	SHAPE (20, 480, 640, 3)


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


iteration : 2/250	SHAPE (20, 480, 640, 3)
iteration : 3/250	SHAPE (20, 480, 640, 3)
iteration : 4/250	SHAPE (20, 480, 640, 3)
iteration : 5/250	SHAPE (20, 480, 640, 3)
iteration : 6/250	SHAPE (20, 480, 640, 3)
iteration : 7/250	SHAPE (20, 480, 640, 3)
iteration : 8/250	SHAPE (20, 480, 640, 3)
iteration : 9/250	SHAPE (20, 480, 640, 3)
iteration : 10/250	SHAPE (20, 480, 640, 3)
iteration : 11/250	SHAPE (20, 480, 640, 3)
iteration : 12/250	SHAPE (20, 480, 640, 3)
iteration : 13/250	SHAPE (20, 480, 640, 3)
iteration : 14/250	SHAPE (20, 480, 640, 3)
iteration : 15/250	SHAPE (20, 480, 640, 3)
iteration : 16/250	SHAPE (20, 480, 640, 3)
iteration : 17/250	SHAPE (20, 480, 640, 3)
iteration : 18/250	SHAPE (20, 480, 640, 3)
iteration : 19/250	SHAPE (20, 480, 640, 3)
iteration : 20/250	SHAPE (20, 480, 640, 3)
iteration : 21/250	SHAPE (20, 480, 640, 3)
iteration : 22/250	SHAPE (20, 480, 640, 3)
iteration : 23/250	SHAPE (20, 480, 640, 3)
iteration : 24/250	SHAPE (20, 480, 640, 3)
iteration : 25/250	

iteration : 191/250	SHAPE (20, 480, 640, 3)
iteration : 192/250	SHAPE (20, 480, 640, 3)
iteration : 193/250	SHAPE (20, 480, 640, 3)
iteration : 194/250	SHAPE (20, 480, 640, 3)
iteration : 195/250	SHAPE (20, 480, 640, 3)
iteration : 196/250	SHAPE (20, 480, 640, 3)
iteration : 197/250	SHAPE (20, 480, 640, 3)
iteration : 198/250	SHAPE (20, 480, 640, 3)
iteration : 199/250	SHAPE (20, 480, 640, 3)
iteration : 200/250	SHAPE (20, 480, 640, 3)
iteration : 201/250	SHAPE (20, 480, 640, 3)
iteration : 202/250	SHAPE (20, 480, 640, 3)
iteration : 203/250	SHAPE (20, 480, 640, 3)
iteration : 204/250	SHAPE (20, 480, 640, 3)
iteration : 205/250	SHAPE (20, 480, 640, 3)
iteration : 206/250	SHAPE (20, 480, 640, 3)
iteration : 207/250	SHAPE (20, 480, 640, 3)
iteration : 208/250	SHAPE (20, 480, 640, 3)
iteration : 209/250	SHAPE (20, 480, 640, 3)
iteration : 210/250	SHAPE (20, 480, 640, 3)
iteration : 211/250	SHAPE (20, 480, 640, 3)
iteration : 212/250	SHAPE (20, 480, 640, 3)
iteration : 213/250	SHAPE (20, 4

In [31]:
train.shape

(250, 20, 258)

### 3.4 split the data (training,testing)

In [32]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [33]:
labels = to_categorical(y)

In [36]:
X_train, X_test, Y_train, Y_test = train_test_split(train, labels, test_size=0.1)

In [37]:
print(
X_train.shape,
X_test.shape,
Y_train.shape,
Y_test.shape,
)

(225, 20, 258) (25, 20, 258) (225, 5) (25, 5)


# 4 - build the model

In [38]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense,Input
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import TensorBoard

log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)



In [40]:
input_layer = Input(shape=(20,258))
layer = LSTM(64,return_sequences=True,activation="relu")(input_layer)
layer = LSTM(128,return_sequences=True,activation="relu")(layer)
layer = LSTM(64,return_sequences=False,activation="relu")(layer)
layer = Dense(64,activation="relu")(layer)
layer = Dense(32,activation="relu")(layer)
layer = Dense(len(actions),activation="softmax")(layer)

model = Model(inputs=input_layer,outputs=layer)

In [41]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 20, 258)]         0         
                                                                 
 lstm_3 (LSTM)               (None, 20, 64)            82688     
                                                                 
 lstm_4 (LSTM)               (None, 20, 128)           98816     
                                                                 
 lstm_5 (LSTM)               (None, 64)                49408     
                                                                 
 dense_3 (Dense)             (None, 64)                4160      
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 5)                 165 

In [42]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [43]:
model.fit(X_train, Y_train, epochs=1000, callbacks=[tb_callback],validation_data=(X_test,Y_test))

Epoch 1/1000
8/8 [==============================] - 3s 89ms/step - loss: 1.5850 - categorical_accuracy: 0.2133 - val_loss: 1.4614 - val_categorical_accuracy: 0.4800
Epoch 2/1000
8/8 [==============================] - 0s 47ms/step - loss: 1.4271 - categorical_accuracy: 0.3778 - val_loss: 3.0159 - val_categorical_accuracy: 0.2800
Epoch 3/1000
8/8 [==============================] - 0s 47ms/step - loss: 1.5205 - categorical_accuracy: 0.4578 - val_loss: 1.4203 - val_categorical_accuracy: 0.3600
Epoch 4/1000
8/8 [==============================] - 0s 50ms/step - loss: 1.2898 - categorical_accuracy: 0.5156 - val_loss: 1.4298 - val_categorical_accuracy: 0.3200
Epoch 5/1000
8/8 [==============================] - 0s 46ms/step - loss: 1.1601 - categorical_accuracy: 0.5111 - val_loss: 1.0612 - val_categorical_accuracy: 0.4800
Epoch 6/1000
8/8 [==============================] - 0s 47ms/step - loss: 1.1586 - categorical_accuracy: 0.5467 - val_loss: 1.1873 - val_categorical_accuracy: 0.4800
Epoch 7/10

8/8 [==============================] - 0s 51ms/step - loss: 0.1921 - categorical_accuracy: 0.9244 - val_loss: 0.4487 - val_categorical_accuracy: 0.8400
Epoch 51/1000
8/8 [==============================] - 0s 48ms/step - loss: 0.1621 - categorical_accuracy: 0.9244 - val_loss: 0.5197 - val_categorical_accuracy: 0.8400
Epoch 52/1000
8/8 [==============================] - 0s 52ms/step - loss: 0.1170 - categorical_accuracy: 0.9689 - val_loss: 0.3630 - val_categorical_accuracy: 0.8400
Epoch 53/1000
8/8 [==============================] - 0s 47ms/step - loss: 0.0956 - categorical_accuracy: 0.9556 - val_loss: 0.7832 - val_categorical_accuracy: 0.8000
Epoch 54/1000
8/8 [==============================] - 0s 49ms/step - loss: 1.0166 - categorical_accuracy: 0.7778 - val_loss: 0.7127 - val_categorical_accuracy: 0.8000
Epoch 55/1000
8/8 [==============================] - 0s 50ms/step - loss: 0.7307 - categorical_accuracy: 0.7644 - val_loss: 0.8532 - val_categorical_accuracy: 0.8000
Epoch 56/1000
8/8 

8/8 [==============================] - 0s 49ms/step - loss: 0.2983 - categorical_accuracy: 0.9067 - val_loss: 0.8104 - val_categorical_accuracy: 0.7600
Epoch 100/1000
8/8 [==============================] - 0s 49ms/step - loss: 0.2571 - categorical_accuracy: 0.9289 - val_loss: 0.4267 - val_categorical_accuracy: 0.8800
Epoch 101/1000
8/8 [==============================] - 0s 49ms/step - loss: 0.5497 - categorical_accuracy: 0.7956 - val_loss: 0.9148 - val_categorical_accuracy: 0.6800
Epoch 102/1000
8/8 [==============================] - 0s 46ms/step - loss: 0.5020 - categorical_accuracy: 0.8311 - val_loss: 0.4813 - val_categorical_accuracy: 0.8800
Epoch 103/1000
8/8 [==============================] - 0s 51ms/step - loss: 0.2066 - categorical_accuracy: 0.9467 - val_loss: 0.4323 - val_categorical_accuracy: 0.8000
Epoch 104/1000
8/8 [==============================] - 0s 52ms/step - loss: 0.1357 - categorical_accuracy: 0.9644 - val_loss: 0.3622 - val_categorical_accuracy: 0.9200
Epoch 105/100

Epoch 148/1000
8/8 [==============================] - 0s 50ms/step - loss: 1.1259e-04 - categorical_accuracy: 1.0000 - val_loss: 0.4365 - val_categorical_accuracy: 0.8800
Epoch 149/1000
8/8 [==============================] - 0s 48ms/step - loss: 1.1011e-04 - categorical_accuracy: 1.0000 - val_loss: 0.6164 - val_categorical_accuracy: 0.8800
Epoch 150/1000
8/8 [==============================] - 0s 47ms/step - loss: 1.6954e-04 - categorical_accuracy: 1.0000 - val_loss: 0.5289 - val_categorical_accuracy: 0.8800
Epoch 151/1000
8/8 [==============================] - 0s 49ms/step - loss: 8.9759e-05 - categorical_accuracy: 1.0000 - val_loss: 0.2075 - val_categorical_accuracy: 0.8800
Epoch 152/1000
8/8 [==============================] - 0s 46ms/step - loss: 9.2914e-05 - categorical_accuracy: 1.0000 - val_loss: 0.1303 - val_categorical_accuracy: 0.8800
Epoch 153/1000
8/8 [==============================] - 0s 47ms/step - loss: 9.2561e-05 - categorical_accuracy: 1.0000 - val_loss: 0.1665 - val_cat

KeyboardInterrupt: 

In [44]:
model.save_weights("model.h5")

In [45]:
model.load_weights('model.h5')

# 5 - Make predictions

### this doesn't make any sence

In [46]:
model.evaluate(X_test,Y_test)

1/1 [==============================] - 0s 22ms/step - loss: 0.7528 - categorical_accuracy: 0.8800


[0.7527683973312378, 0.8799999952316284]

In [47]:
model.evaluate(X_train,Y_train)

8/8 [==============================] - 0s 9ms/step - loss: 7.8693e-06 - categorical_accuracy: 1.0000


[7.869308319641277e-06, 1.0]

In [48]:
x[0]

'../../data/train/signer0_sample431_color.mp4'

# 6 - test in real time

In [57]:
model.predict(np.array([train[0]]))

array([[9.9999952e-01, 2.4448259e-11, 3.1743037e-18, 4.4404717e-07,
        1.1752664e-15]], dtype=float32)

In [58]:
from scipy import stats

colors = [(245,117,16), (117,245,16), (16,117,245),(255,0,0),(0,255,0)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame


In [59]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()
        if(not ret):
            break
        frame = cv2.resize(frame, (640, 480))

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-20:]
        
        if len(sequence) == 20:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-2:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

In [50]:
cap.release()

In [51]:
cv2.destroyAllWindows()